# Generators 
- another way to define an iterable
- a generator is defined by using a 'yield' statement inside a 'def'
- executing the function returns the generator
- calling 'next' on a generator will cause the generator to execute until it 
hits a 'yield' statement. the arg supplied to 'yield' will be returned by 'next'. the next time 'next' is called on the generator, the generator will resume executing on the statement following the yield. 
- all local variable values are preserved between between 'next' calls to the generator
- falling off the end of the function, or executing a 'return' statement, will terminate the generator.
- once a generator terminates, it is 'exhausted', and can not be used again
- the generator signals exhaustion by raising 'StopIteration' on a 'next' call


In [5]:
# a simple function definition

def foo(n):
    x = n+1
    y = n+2
    print(x)
    print(y)

# execute the function
foo(4)

5
6


In [12]:
# if a def has a yield statement,
# it does something very different -
# it defines a generator

def bar(n):
    x = n+1
    y = n+2
    print(x)
    yield(x)
    print(x+y)
    yield(x+y)
    
# build the generator

g = bar(4)

In [13]:
# execute the generator by using the iterator protocol
# the value of x, 5, was printed, then was 'yielded' as the 
# return value of the next call. 
# the generator code paused execution just after the yield(x)
# call

next(g)

5


5

In [14]:
# now the generator continues execution, from 
# just after the first yield(x)
# note the values of x and y have been remembered. 

next(g)

11


11

In [15]:
# now there no yields left thus no more values can be returned
# from calling next, so the generator raises a StopIteration error
# the generator is now spent, and should be discarded

next(g)

StopIteration: 

In [18]:
# zap is bar with the prints removed

def zap(n):
    x = n+1
    y = n+2
    yield(x)
    yield(x+y)
   

In [19]:
# use 'for' to iterate the generator

for j in zap(4):
    print(j)

5
11


In [20]:
# or make a list

list(zap(4))

[5, 11]

In [21]:
# or do a list comprehension

[j+10 for j in zap(4)]

[15, 21]

# yield from statement

In [31]:
# generator for chars in a string

def chars(s):
    for c in s:
        yield c
        
cs = chars('larry')
for c in cs:
    print(c)
    

l
a
r
r
y


In [32]:
# 'yield from' will yield everything from its generator argument

def gfrom(g):
    # when g is exhausted, move to next statement
    yield from g
    yield 'another yield'
    
gs = gfrom(chars('larry'))

for c in gs:
    print(c)

l
a
r
r
y
another yield


# generators vs classes
- both preserve 'state' information, in different ways
- generators
    - save local variable bindings and program execution location
    - automatically define an iterator
- classes
    - save object attribute values
    - iterator must be defined explicitly

# compare by implementing fibonacci series both ways

- fibonacci series is 1,1,2,3,5,8...
- f(0) = 1
- f(1) = 1
- f(n) = f(n-1) + f(n-2)

# fibonaaci generator

In [34]:
def fibg():
    # easy way to handle the first two ones
    yield(1)
    yield(1)
    last = 1
    last2 = 1
    while True:
        sum = last + last2
        yield(sum)
        last2 = last
        last = sum

for g,j in zip(fibg(), range(10)):
    print(g)


1
1
2
3
5
8
13
21
34
55


# fibonacci class
- must explicitly define iteration with ```__iter__ and __next__``` methods
- must explicitly save state on object attributes


In [36]:
# implement the iteration protocol, 
# using the `__iter__ and __next__' methods

class fibc:
    def __init__(self):
        # state we will save in object attributes
        self.old = 1
        self.older = 1
    
    # return the iterator for the obj, 
    # which is the object itself
    def __iter__(self):
        return self
    
    # returns the next element in the iteration
    # since this sequence is infinite, we never 
    # throw the StopIteration error
    def __next__(self):
        (self.old, self.older, rtn) = (self.old + self.older, self.old, self.older)
        return rtn 

# note the generator and class can be used the same way,
# because they both implement the iteration protocol

for g,j in zip(fibc(), range(10)):
    print(g)

1
1
2
3
5
8
13
21
34
55


# conclusion
- for fibonaaci, the generator approach is much simpler
- however, classes are much more general and can be used in ways that generators do not support

# Generator Expression
- an expression that evaluates to a generator
- looks like a list comprehension, but with outer '()' instead of '[]'

In [33]:
def ge(n):
    # can't put a 'def' in a return, but
    # an expression is ok
    return ( j**2 for j in range(2, n) if j != 3)

g = ge(8)

In [34]:
# pick first two manually

next(g)

4

In [35]:
# skipped j == 3

next(g)

16

In [36]:
# for gets the rest

for j in g:
    print(j)

25
36
49


# A generator can represent an infinite sequence (sort of)
- eager approach can't work - not possible to make a list of ALL the even integers
- but in some sense lazy approach can represent that list with a generator, by supplying as many as are asked for

In [22]:
def infinite(start, incr):
    e = start
    # this generator will never terminate
    while True:
        yield(e)
        e += incr

In [23]:
# eg represents the positive even numbers
# but you can't use constructs like 'for', or list 
# comprehensions on infinite generators because 
# the loops will never terminate
# here the range terminates the loop

eg = infinite(2,2)
[next(eg) for j in range(5)]

[2, 4, 6, 8, 10]

In [25]:
# another way to limit the iteration
# zip will stop when the range is exhausted

[g+10 for g,z in zip(infinite(2,2), range(5))]

[12, 14, 16, 18, 20]

In [24]:
# a generator can use another generator

def evenPowersOf2():
    eg = infinite(2,2)
    while True:
        e = next(eg)
        yield 2**e

ep2 = evenPowersOf2()
[next(ep2) for j in range(5)]

[4, 16, 64, 256, 1024]

In [23]:
[next(ep2) for j in range(5)]

[4096, 16384, 65536, 262144, 1048576]

# can do operations on infinite series

In [24]:
import operator

# add infinite series

eg = infinite(2,2)
g5 = infinite(5,5)

def opgen(op, g1, g2):
    while True: 
        e1 = next(g1)
        e2 = next(g2)
        yield op(e1,e2)
        
og = opgen(operator.add, eg, g5)

[next(og) for j in range(5)]


[7, 14, 21, 28, 35]

In [25]:
# subtract infinite series

eg = infinite(2,2)
g5 = infinite(5,5)
og = opgen(operator.sub, eg, g5)

[next(og) for j in range(5)]

[-3, -6, -9, -12, -15]

# Modifying a Running Generator
- can change generator state at any time

In [27]:
def counter(maximum):
    cnt = 0
    while cnt < maximum:
        # peculiar syntax
        val = (yield cnt)
        # If value provided, change counter
        if val is not None:
            cnt = val
        else:
            cnt += 1


In [28]:
c = counter(1000)
[next(c) for j in range(10)]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [29]:
# change the 'cnt' variable that the generator saves 
# '(yield cnt)' in generator will return 300

c.send(300)

# generator continues from new value
[next(c) for j in range(10)]

[301, 302, 303, 304, 305, 306, 307, 308, 309, 310]

In [30]:
# the generator is nowhere near done, 
# but we can terminate it

c.close()

In [32]:
# the generator is exhausted now

next(c)

StopIteration: 

# suppose want to sum 1,000,000 squares
- try three different approaches

In [26]:
# 1

# could do

mil = 1000**2

sq = [x**2 for x in range(mil)]

# built in function that will sum a list
sum(sq)


333332833333500000

In [27]:
# 2 
# accumulation variable and for loop

total = 0 
for x in range(mil):
    total += x**2
total

333332833333500000

In [28]:
# 3
# use a generator

# which way is better? worst?

sum(x**2 for x in range(mil))

333332833333500000